<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2022/blob/main/Seminars/lab_03_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
from sklearn.datasets import make_classification
%pylab inline
import seaborn as sns
sns.set_style('whitegrid')
pylab.rcParams['figure.figsize'] = (12.0, 5.0)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.preprocessing import PolynomialFeatures
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

## Linear SVM reminder

Let's look at binary classification problem. Training samples are given by $\{(x_n, y_n)\}_{n=1}^N$, where $N$ — number of objects, $\boldsymbol x_n \in \mathbb{R}^d$ — feature vector of object, $y_n \in \{+1, -1\}$ — class of object.

SVM trains model for separating hyperplane:
$$f(\boldsymbol x) = \boldsymbol w^T \boldsymbol x + b$$
Parameters of model — vector of weights $\boldsymbol w \in \mathbb{R}^d$ and bias $b \in \mathbb{R}$.

Training is done by solving optimisation problem:
$$
\begin{gather}
    \frac{1}{2} \| \boldsymbol w \|^2 + C \sum_{n=1}^N \xi_n \to \min_{\boldsymbol w, \boldsymbol \xi, b} \\
    \text{s.t.: } \quad y_n (\boldsymbol w^T \boldsymbol x_n + b) \geq 1 - \xi_n, \quad \xi_n \geq 0, \quad \forall n=1,\dots,N
\end{gather}
$$

Constraint $y_n (\boldsymbol w^T \boldsymbol x_n + b) \geq 1$ assures that objects are correctly classified by separating hyperplane. Since in practice the sample could not be linearly separable the slack variables $\xi_n$ are introduced , which weakens condition of right classification. $\| \boldsymbol w \|^2$ penalise small width of margin,  $\sum_n \xi_n$ penalise weakens of constraints. 

The solution of optimisation problem is given by $(\boldsymbol w_{\star}, \boldsymbol \xi_{\star}, b_{\star})$, some of the constraints become active, i.e. become a exact equality:
$$\quad y_n (\boldsymbol w_{\star}^T \boldsymbol x_n + b_{\star}) = 1 - \xi_{\star,n}$$
Objects, corresponding to active constraints called $\textbf{support vectors}$.


Hyperparameter $C$ is responsible of balancing the width of margin and errors, made by classifier. It shows the generalizing property of the separating hyperplane - big values of $C$ corresponds to less generalizing ability and can lead to overfitting, if the data is well described by linear model. To select $C$ one must do cross-validation on validation set to find the best value.

### Realisation

There are two realisations of linear SVM in sklearn : [LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) and [SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) with *linear* kernel. They build on different libraries, with solve optimisation problem *liblinear* in first case and *libsvm* in second.

Here we will use [sklearn.svm.SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) with *kernel='linear'*.

We generate data samples with:
- linearly separable classes
- with well separable classes, but not linearly
- with non separable classes.

### The function below provides imaging and data samples creatinon. Your task is to use it according to the next exercises.

In [ ]:
# From sklearn SVC examples
def svc_plotter(classifiers, name, poly_f=False, polynomial_expand=None):
    h = .02  # step size in the mesh
        
    X_sep, y_sep = make_blobs(n_samples=400, n_features=2, centers=2, cluster_std=1.5, random_state=42)
    linearly_separable = (X_sep, y_sep)

    X, y = make_classification(n_samples=400, n_features=2, n_redundant=0, n_informative=2,
                               random_state=231, n_clusters_per_class=1)

    rng = np.random.RandomState(2)
    X += 15 * rng.uniform(size=X.shape)
    linearly_bad_separable = (X, y)


    datasets = [linearly_separable,
                linearly_bad_separable,
                make_circles(n_samples=400, noise=0.05, random_state=42)]

    # iterate over datasets
    f, axes = plt.subplots(len(datasets), len(classifiers), figsize=(20, 20))
    for ds_cnt, ds in enumerate(datasets):
        # preprocess dataset, split into training and test part
        X, y = ds
        X = StandardScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=.4, random_state=42)

        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        # iterate over classifiers
        index = 0
        for clf in classifiers:
            if poly_f:
                clf.fit(polynomial_expand.fit_transform(X_train), y_train)
            else:
                clf.fit(X_train, y_train)
            ax = axes[ds_cnt, index]
            index += 1

            # Plot the decision boundary. For that, we will assign a color to each
            # point in the mesh [x_min, x_max]x[y_amin, y_max].
            if hasattr(clf, "decision_function"):
                if poly_f:
                    Z = clf.decision_function(polynomial_expand.transform(np.c_[xx.ravel(), yy.ravel()]))
                else:
                    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
            else:
                Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
            # Put the result into a color plot
            Z = Z.reshape(xx.shape)
            #ax.pcolormesh(xx, yy, Z > 0, cmap=plt.cm.Paired)
            ax.contourf(xx, yy, Z, cmap=cm.RdBu, alpha=.8, linestyles=['--', '-', '--'],levels=[-1, 0, 1])
            ax.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], levels=[-1, 0, 1])

            #support vectors
#             ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80,
#                     facecolors='none', zorder=10)

            if poly_f:
                poly_features= polynomial_expand.fit_transform(X_train)
                train_classes = [poly_features[~y_train.astype(bool)], poly_features[y_train.astype(bool)]] 
            else:
                train_classes = [X_train[~y_train.astype(bool)], X_train[y_train.astype(bool)]] 
            if not poly_f:
                ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=180,
                         facecolors='none', zorder=10, edgecolors='black', linewidths=0.5)
            # Plot also the training points
            ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, alpha=0.3, s=36)
            # and testing points
            ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                       alpha=0.5, s=36)

            ax.set_xlim(xx.min(), xx.max())
            ax.set_ylim(yy.min(), yy.max())
            ax.set_xlabel("x_1")
            ax.set_ylabel("x_2")
            ax.set_title(name[index - 1])

### Task 1

Vary parameter C(in log scale [hint.nplogspace) and provide a list of SVMs with linear kernel to function. name is list of strings, denoting different labels(values of c)

In [ ]:
classifiers = []
name = []
# Hint: you should add new SVC classifiers to list and come up with the name for each of them. Add the name to the other list)
for C in <Hint>: #your code
  # YOUR CODE HERE

svc_plotter(classifiers, name)

### Task 2

How do the number of support vectors depends on C on different datasets?
Extract info about support vectors from SVC and plot a graph: Number of Support Vectors VS value of C for all 3 datasets declared below.

In [ ]:
X_sep, y_sep = make_blobs(n_samples=400, n_features=2, centers=2, cluster_std=1.5, random_state=42)
linearly_separable = (X_sep, y_sep)

X, y = make_classification(n_samples=400, n_features=2, n_redundant=0, n_informative=2,
                           random_state=231, n_clusters_per_class=1)

rng = np.random.RandomState(2)
X += 15 * rng.uniform(size=X.shape)
linearly_bad_separable = (X, y)


datasets = [linearly_separable,
            linearly_bad_separable,
            make_circles(n_samples=400, noise=0.05, random_state=42)]

In [ ]:
vectors = [[],[],[]]
paramd_grid = ?# <your code>
for index, dataset in enumerate(datasets):
    # YOUR CODE HERE
    # Create a list of classifiers
    for svc in <your list>:
        svc.fit(dataset[0], dataset[1])
        vectors[index].append(WHAT?)# <your code>

In [ ]:
plt.figure
plt.plot(paramd_grid, vectors[0], label = 'Dataset_1')
plt.plot(paramd_grid, vectors[1], label = 'Dataset_2')
plt.plot(paramd_grid, vectors[2], label = 'Dataset_3')
plt.xscale('log')
plt.xlabel('C(log scale)')
plt.ylabel('Number of support vectors')
plt.ylim(0, 410)
plt.title("Dependance of number of support vectors from C")
plt.legend()

## Task 3

As you can see, for dataset 3, there is no linear hyperplane(e.g. classes can't be separated with a hyperplane)
, thus, to solve this problem, one can change feature space, in which the linear model describe dataset better.

$$\boldsymbol x \in \mathbb{R}^d \mapsto \phi(\boldsymbol x) \in \mathbb{R}^t$$

For example, adding all pairwise product of features: $\phi(x_1, \dots, x_d) = (x_1, \dots, x_d, x_1^2, x_1x_2, \dots, x_d^2)$ transform to the space, where linear hyperplane is a quadratic form in initial feature space.

[Video with demonstration](https://youtu.be/9NrALgHFwTo)
<img src="https://www.aionlinecourse.com/uploads/tutorials/2019/07/11_Kernel_SVM_5.jpg" width=50%> 



There is a module in sklearn which will can it for you, and not only. Have a look at it.

Use $\textbf{sklearn.preprocessing PolynomialFeatures(with include_bias=False)}$ and plot the results as above. Briefly describe the results

In [ ]:
classifiers = []
name = []
# YOUR CODE HERE
for C in ?:
  # YOUR CODE HERE (Hint: it is very similar to the frist task)
svc_plotter(classifiers, name, poly_f=True, polynomial_expand=<YOUR instance of PolynomialFeatures>)

SyntaxError: invalid syntax (<ipython-input-33-fd92367e7dca>, line 4)

We transform out feature space to quadratic feature space where our classes are linearly separable

## Kernel SVM

![](http://i.imgur.com/bJAzRCt.png)

Linaer SVM problem, covered above, is usually called direct optimisation problem. Any direct problem has $\textbf{dual}$ problem and in some cases optimums of both problems coincide.


Dual problem for SVM is:
$$
\begin{gather}
    \sum_{n} \alpha_n - \frac{1}{2}\sum_{n}\sum_{n'} \alpha_{n}\alpha_{n'} y_{n}y_{n'} x_{n}^Tx_{n'} \to \max_{\alpha} \\
    \begin{aligned}
        \text{s.t. } \quad  
        & 0 \le \alpha_n \le C, \quad \forall n = 1, \dots, N \\
        & \sum_{n} \alpha_n y_n = 0
    \end{aligned}
\end{gather}
$$

Vector of dual variables is being optimised $\alpha_n$. Object $x_n$ is a SV, if $\alpha_n > 0$.

The predicted label is given by:
$$\hat{y}(x) = \text{sign}\left(\sum_{n}\alpha_{n}y_{n}x^Tx_{n} + b\right).$$

#### Kernel trick
Notice, that dual problem has features only as a scalar product $x^Tx'$. This observation helps us to perform kernel trick - implicitly change feature space. Instead of calculating $\phi(\boldsymbol x)$ (as we did before) we will compute scalar product $k(\boldsymbol x, \boldsymbol x')$ called $\textbf{kernel}$ and plug it insted of $x^Tx'$ above. 

### Task 4

Try different SVM kernel and plot pictures as above, look what other parameters kernels have and how they affect results.
- polynomial: $k(x, x') = (x^Tx' + 1)^d$ with different $d = 2,3,\dots$
- Gaussian RBF: $k(x, x') = \exp(-\sigma\|x - x'\|^2)$

## Poly kernel

### degree=2

In [ ]:
classifiers = []
name = []
# YOUR CODE HERE
svc_plotter(classifiers, name)

### degree=5

In [ ]:
classifiers = []
name = []
# YOUR CODE HERE
svc_plotter(classifiers, name)

### degree=10

In [ ]:
classifiers = []
name = []
# YOUR CODE HERE
svc_plotter(classifiers, name)

## RBF kernel(vary gamma parameter in logscale)

In [ ]:
classifiers = []
name = []
# YOUR CODE HERE
svc_plotter(classifiers, name)

Video with RBF kernel simply explaned https://youtu.be/Qc5IyLW_hns